<a href="https://colab.research.google.com/github/vpysmennyi/machinelearning-learning/blob/main/hometask5_CIFAR100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CIFAR100

In [15]:
import numpy as np
import torch
import torchvision.transforms as transforms 

from torch import nn
from torch.utils.data import DataLoader
from torch.optim import SGD
from torchvision.datasets import CIFAR100

train_ds = CIFAR100('.', train=True, transform=transforms.ToTensor(), download=True)
val_ds = CIFAR100('.', train=False, transform=transforms.ToTensor(), download=True)

batch_size = 256
hidden_size = 256

train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

class BatchNorm(nn.Module):
  def __init__(self, n_features, momentum=0.1, eps=1e-5):
    super(BatchNorm, self).__init__()
    self.momentum = momentum
    self.eps = eps
    
    # afine parameters
    self.weight = nn.Parameter(torch.Tensor(n_features))
    self.bias = nn.Parameter(torch.Tensor(n_features))

    # running statistics
    self.register_buffer('running_mean', torch.zeros(n_features))
    self.register_buffer('running_var', torch.zeros(n_features))

    self.reset_parameters()

  def reset_parameters(self):
    nn.init.ones_(self.weight)
    nn.init.zeros_(self.bias)
    self.running_mean.zero_()
    self.running_var.fill_(1)

  def forward(self, x):
    if self.training:
      batch_mean = torch.mean(x, dim=0)
      batch_var = torch.var(x, dim=0)
      
      x = (x - batch_mean) / torch.sqrt(batch_var + self.eps)

      self.running_mean = self.momentum * batch_mean.detach() + (1 - self.momentum) * self.running_mean
      self.running_var = self.momentum * batch_var.detach() + (1 - self.momentum) * self.running_var
    else:
      x = (x - self.running_mean) / torch.sqrt(self.running_var + self.eps)

    x = self.weight * x + self.bias
    return x

class FFN(nn.Module):
  def __init__(self):
    super(FFN, self).__init__()

    self.net = nn.Sequential(
        nn.Flatten(),
        nn.Linear(32*32*3, hidden_size),
        nn.ReLU(),
        BatchNorm(hidden_size),
        nn.Linear(hidden_size, hidden_size),
        nn.ReLU(),
        BatchNorm(hidden_size),
        nn.Linear(hidden_size, hidden_size),
        nn.ReLU(),
        BatchNorm(hidden_size),
        nn.Linear(hidden_size, hidden_size),
        nn.ReLU(),
        BatchNorm(hidden_size),
        nn.Linear(hidden_size, hidden_size//2),
        nn.ReLU(),
        BatchNorm(hidden_size//2),
        nn.Linear(hidden_size//2, hidden_size//2),
        nn.ReLU(),
        BatchNorm(hidden_size//2),
        nn.Linear(hidden_size//2, 100)
    )

  def forward(self, x):
    return self.net(x)

Files already downloaded and verified
Files already downloaded and verified


In [19]:
torch.cuda.init()
network = FFN()
opt = SGD(network.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()
n_epochs = 40

for epoch_ind in range(n_epochs):
  train_loss, train_acc = [], []
  
  network.train()
  for (X, Y) in train_dl:
    opt.zero_grad()
    Yp = network(X)
    loss = loss_fn(Yp, Y)
    loss.backward()
    opt.step()

    accuracy = (Yp.argmax(1) == Y).float().mean()
    train_acc.append(accuracy.detach())
    train_loss.append(loss.detach())

  print(f"{epoch_ind} Train loss: {torch.stack(train_loss).mean():.6f}\t acc: {torch.stack(train_acc).mean():.4f}")

  network.eval()
  val_loss, val_acc = [], []

  for (X, Y) in val_dl:
    Yp = network(X)
    
    accuracy = (Yp.argmax(1) == Y).float().mean()
    val_acc.append(accuracy)

    loss = loss_fn(Yp, Y)
    val_loss.append(loss)

  print(f"{epoch_ind} Val loss: {torch.stack(val_loss).mean():.6f}\t acc: {torch.stack(val_acc).mean():.4f}")

0 Train loss: 4.679153	 acc: 0.0169
0 Val loss: 4.600177	 acc: 0.0270
1 Train loss: 4.515767	 acc: 0.0380
1 Val loss: 4.470586	 acc: 0.0449
2 Train loss: 4.413090	 acc: 0.0555
2 Val loss: 4.370447	 acc: 0.0640
3 Train loss: 4.327718	 acc: 0.0746
3 Val loss: 4.285659	 acc: 0.0834
4 Train loss: 4.252253	 acc: 0.0890
4 Val loss: 4.217187	 acc: 0.0940
5 Train loss: 4.190385	 acc: 0.1002
5 Val loss: 4.153252	 acc: 0.1068
6 Train loss: 4.133501	 acc: 0.1097
6 Val loss: 4.092214	 acc: 0.1176
7 Train loss: 4.084186	 acc: 0.1192
7 Val loss: 4.056239	 acc: 0.1239
8 Train loss: 4.042939	 acc: 0.1269
8 Val loss: 4.005868	 acc: 0.1357
9 Train loss: 4.000931	 acc: 0.1347
9 Val loss: 3.978912	 acc: 0.1379
10 Train loss: 3.962328	 acc: 0.1408
10 Val loss: 3.922088	 acc: 0.1464
11 Train loss: 3.929048	 acc: 0.1458
11 Val loss: 3.900117	 acc: 0.1519
12 Train loss: 3.898016	 acc: 0.1507
12 Val loss: 3.867323	 acc: 0.1569
13 Train loss: 3.866975	 acc: 0.1549
13 Val loss: 3.839653	 acc: 0.1611
14 Train los

0 Train loss: 4.679153	 acc: 0.0169
0 Val loss: 4.600177	 acc: 0.0270
1 Train loss: 4.515767	 acc: 0.0380
1 Val loss: 4.470586	 acc: 0.0449
2 Train loss: 4.413090	 acc: 0.0555
2 Val loss: 4.370447	 acc: 0.0640
3 Train loss: 4.327718	 acc: 0.0746
3 Val loss: 4.285659	 acc: 0.0834
4 Train loss: 4.252253	 acc: 0.0890
4 Val loss: 4.217187	 acc: 0.0940
5 Train loss: 4.190385	 acc: 0.1002
5 Val loss: 4.153252	 acc: 0.1068
6 Train loss: 4.133501	 acc: 0.1097
6 Val loss: 4.092214	 acc: 0.1176
7 Train loss: 4.084186	 acc: 0.1192
7 Val loss: 4.056239	 acc: 0.1239
8 Train loss: 4.042939	 acc: 0.1269
8 Val loss: 4.005868	 acc: 0.1357
9 Train loss: 4.000931	 acc: 0.1347
9 Val loss: 3.978912	 acc: 0.1379
10 Train loss: 3.962328	 acc: 0.1408
10 Val loss: 3.922088	 acc: 0.1464
11 Train loss: 3.929048	 acc: 0.1458
11 Val loss: 3.900117	 acc: 0.1519
12 Train loss: 3.898016	 acc: 0.1507
12 Val loss: 3.867323	 acc: 0.1569
13 Train loss: 3.866975	 acc: 0.1549
13 Val loss: 3.839653	 acc: 0.1611
14 Train loss: 3.837757	 acc: 0.1607
14 Val loss: 3.819637	 acc: 0.1630
15 Train loss: 3.809928	 acc: 0.1656
15 Val loss: 3.791339	 acc: 0.1695
16 Train loss: 3.784164	 acc: 0.1691
16 Val loss: 3.764105	 acc: 0.1721
17 Train loss: 3.757521	 acc: 0.1736
17 Val loss: 3.738889	 acc: 0.1788
18 Train loss: 3.735371	 acc: 0.1787
18 Val loss: 3.712357	 acc: 0.1812
19 Train loss: 3.710176	 acc: 0.1805
19 Val loss: 3.684084	 acc: 0.1856
20 Train loss: 3.688774	 acc: 0.1846
20 Val loss: 3.659141	 acc: 0.1890
21 Train loss: 3.667568	 acc: 0.1886
21 Val loss: 3.644920	 acc: 0.1913
22 Train loss: 3.644706	 acc: 0.1920
22 Val loss: 3.628203	 acc: 0.1962
23 Train loss: 3.623743	 acc: 0.1943
23 Val loss: 3.618592	 acc: 0.1964
24 Train loss: 3.605402	 acc: 0.1972
24 Val loss: 3.593551	 acc: 0.2018
25 Train loss: 3.586753	 acc: 0.2022
25 Val loss: 3.579695	 acc: 0.2034
26 Train loss: 3.566078	 acc: 0.2044
26 Val loss: 3.552220	 acc: 0.2056
27 Train loss: 3.546269	 acc: 0.2098
27 Val loss: 3.534546	 acc: 0.2096
28 Train loss: 3.528364	 acc: 0.2121
28 Val loss: 3.511056	 acc: 0.2115
29 Train loss: 3.510128	 acc: 0.2143
29 Val loss: 3.489616	 acc: 0.2167
30 Train loss: 3.490389	 acc: 0.2189
30 Val loss: 3.469536	 acc: 0.2210
31 Train loss: 3.475932	 acc: 0.2208
31 Val loss: 3.454343	 acc: 0.2238
32 Train loss: 3.458840	 acc: 0.2233
32 Val loss: 3.449950	 acc: 0.2253
33 Train loss: 3.442073	 acc: 0.2252
33 Val loss: 3.459967	 acc: 0.2232
34 Train loss: 3.426214	 acc: 0.2279
34 Val loss: 3.414595	 acc: 0.2334
35 Train loss: 3.407106	 acc: 0.2329
35 Val loss: 3.393553	 acc: 0.2365
36 Train loss: 3.389818	 acc: 0.2369
36 Val loss: 3.375656	 acc: 0.2399
37 Train loss: 3.374552	 acc: 0.2385
37 Val loss: 3.358344	 acc: 0.2391
38 Train loss: 3.359281	 acc: 0.2423
38 Val loss: 3.340178	 acc: 0.2461
39 Train loss: 3.343874	 acc: 0.2448
39 Val loss: 3.337270	 acc: 0.2424